In [ ]:
# Change these variables to the intended area
location_name = 'baltimore' 

from pyzipcode import ZipCodeDatabase

# Initialize the Zip Code Database
zcdb = ZipCodeDatabase()

# Get all zip codes for Maryland and Pennsylvania
md_zip_codes = [z.zip for z in zcdb.find_zip(state="MD")]
pa_zip_codes = [z.zip for z in zcdb.find_zip(state="PA")]

# Convert zip codes to integers (optional, if needed)
zip_codes = [int(z) for z in md_zip_codes] + [int(z) for z in pa_zip_codes]

# Set core CBG to cluster on and the minimum population (USER INPUT)
core_cbg = '245101001001' 
min_cluster_pop = 50000

import geopandas as gpd
import numpy as np

import pandas as pd

import networkx as nx
import json
import pickle

gdf1 = gpd.read_file('tl_2010_24_bg10/tl_2010_24_bg10.shp')
gdf2 = gpd.read_file('tl_2010_42_bg10/tl_2010_42_bg10.shp')
gdf3 = gpd.read_file('tl_2010_40_bg10/tl_2010_40_bg10.shp')
gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2, gdf3], ignore_index=True))

"""


import folium
from folium import plugins
import pandas as pd


# Simplified POI data with lists of CBGs for each location
poi_data = [
    {"safegraph_place_id": "sg:philly1", "cbgs": ["421010001001", "421010001002", "421010002001"], "latitude": 39.9656, "longitude": -75.1804},
    {"safegraph_place_id": "sg:philly2", "cbgs": ["421010003001", "421010003002", "421010004001"], "latitude": 39.9496, "longitude": -75.1503},
    {"safegraph_place_id": "sg:philly3", "cbgs": ["421010005001", "421010005002", "421010006001"], "latitude": 39.9533, "longitude": -75.1599},
    {"safegraph_place_id": "sg:philly4", "cbgs": ["421010007001", "421010007002", "421010008001"], "latitude": 39.9489, "longitude": -75.1494},
    {"safegraph_place_id": "sg:philly5", "cbgs": ["421010009001", "421010009002", "421010010001"], "latitude": 39.9682, "longitude": -75.1720},
    {"safegraph_place_id": "sg:philly6", "cbgs": ["421010011001", "421010011002", "421010012001"], "latitude": 39.9542, "longitude": -75.1593},
    {"safegraph_place_id": "sg:philly7", "cbgs": ["421010013001", "421010013002", "421010014001"], "latitude": 39.9740, "longitude": -75.1952},
    {"safegraph_place_id": "sg:philly8", "cbgs": ["421010015001", "421010015002", "421010016001"], "latitude": 39.9583, "longitude": -75.1721},
    {"safegraph_place_id": "sg:philly9", "cbgs": ["421010017001", "421010017002", "421010018001"], "latitude": 39.9488, "longitude": -75.1714},
    {"safegraph_place_id": "sg:philly10", "cbgs": ["421010019001", "421010019002", "421010020001"], "latitude": 39.9526, "longitude": -75.1638},
]

# Convert the data to a DataFrame
df = pd.DataFrame(poi_data)

# Initialize a folium map centered on Philadelphia
map_philly = folium.Map(location=[39.9526, -75.1652], zoom_start=13)


# List to hold features for animation
features = []
start_time = pd.Timestamp('today')

# Iterate through the POIs, staggering their appearance
for idx, row in df.iterrows():
    for cbg in row['cbgs']:
        shape = gdf[gdf['GEOID10'] == cbg]
        
        if not shape.empty:
            # Convert shape to EPSG:4326 for correct map projection
            shape = shape.to_crs(epsg='4326')
            
            # Create GeoJSON feature
            geojson = json.loads(shape.to_json())
            feature = geojson['features'][0]
            
            # Stagger the appearance of CBGs by days (1 day per POI)
            feature['properties']['times'] = [(start_time + pd.Timedelta(idx, 'D')).isoformat()]
            feature['properties']['style'] = {'fillColor': '#008000', 'color': '#008000', 'fillOpacity': 0.4}
            features.append(feature)

# Add the timestamped GeoJSON layer for the animation
time_geojson = plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT6H', add_last_point=True, auto_play=False, loop=False
)
map_philly.add_child(time_geojson)

# Save the map as an HTML file
map_philly.save('philladelphia.html')

"""



'\n\n\nimport folium\nfrom folium import plugins\nimport pandas as pd\n\n\n# Simplified POI data with lists of CBGs for each location\npoi_data = [\n    {"safegraph_place_id": "sg:philly1", "cbgs": ["421010001001", "421010001002", "421010002001"], "latitude": 39.9656, "longitude": -75.1804},\n    {"safegraph_place_id": "sg:philly2", "cbgs": ["421010003001", "421010003002", "421010004001"], "latitude": 39.9496, "longitude": -75.1503},\n    {"safegraph_place_id": "sg:philly3", "cbgs": ["421010005001", "421010005002", "421010006001"], "latitude": 39.9533, "longitude": -75.1599},\n    {"safegraph_place_id": "sg:philly4", "cbgs": ["421010007001", "421010007002", "421010008001"], "latitude": 39.9489, "longitude": -75.1494},\n    {"safegraph_place_id": "sg:philly5", "cbgs": ["421010009001", "421010009002", "421010010001"], "latitude": 39.9682, "longitude": -75.1720},\n    {"safegraph_place_id": "sg:philly6", "cbgs": ["421010011001", "421010011002", "421010012001"], "latitude": 39.9542, "longi

In [107]:

#Read safegraph files
filename = location_name + '.csv'

try:
    df = pd.read_csv(filename)
except:
    datalist = []

    with pd.read_csv('patterns.csv', chunksize=10000) as reader:
        for chunk in reader:
            datalist.append(chunk[chunk['postal_code'].isin(zip_codes)])

    df = pd.concat(datalist, axis=0)
    del datalist

    try:
        
        df['poi_cbg'] = df['poi_cbg'].astype('int64')
    except:
        print(df['poi_cbg'])
        pass
    df.to_csv(filename)

In [ ]:
"""
cols = ['location_name', 'safegraph_place_id', 'latitude', 'longitude']
filename = location_name + '.pois.csv'

for _, row in df.iterrows():
    if not row['visitor_daytime_cbgs']:
        continue
    
    for visitor_cbg in json.loads(row['visitor_daytime_cbgs']).keys():
        
        if visitor_cbg not in zip_codes:
            zip_codes.append(visitor_cbg)

try:
    poif = pd.read_csv(filename)
    print('aight')
except:
    datalist = []

    with pd.read_csv('2021_05_05_03_core_poi.csv', chunksize=10000) as reader:
        for chunk in reader:
            datalist.append(chunk[chunk['postal_code'].isin(zip_codes)])

    poif = pd.concat(datalist, axis=0)
    del datalist

    poif.to_csv(filename)
"""


aight


In [109]:
#Try to avoid this function especially in loops since it takes time
def cbg_geocode(cbg_code):

    cbg_row = gdf[gdf['GEOID10'] == cbg_code]
    
    # Check if a match was found
    if not cbg_row.empty:
        # Get the centroid of the matched geometry
        centroid = cbg_row.geometry.centroid.iloc[0]
        return centroid.y, centroid.x  # Return latitude, longitude
    else:
        print(f"CBG code {cbg_code} not found in the data.")
        return None

cbg_geocode(core_cbg)

C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1106654488.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = cbg_row.geometry.centroid.iloc[0]


(39.64007694608107, -77.70954929430157)

In [110]:
def poi_geocode(poi):
    '''Finds the latitude and longitude coordinates for a POI

    Args:
        poi (pd.Series): Series that contains the SafeGraph data for a specific POI

    Returns:
        pd.Series: Name, Latitude, & Longitude coords for a POI
    '''
    place_id = poi if type(poi) == str else poi['safegraph_place_id']
    poi_data = poif.loc[poif['safegraph_place_id'] == place_id]
    print(poi_data)
    return pd.Series(data={
        'label': place_id,
        'latitude': poi_data['latitude'],
        'longitude': poi_data['longitude']
    })

cbg_geocode(core_cbg)

C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1106654488.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = cbg_row.geometry.centroid.iloc[0]


(39.64007694608107, -77.70954929430157)

In [111]:
#function to find distances between cbgs, used in the distance graph
from math import sin, cos, atan2, pi, sqrt
def distance(lat1, long1, lat2, long2):
    lat1 = lat1*pi/180
    long1 = long1*pi/180
    lat2 = lat2*pi/180
    long2 = long2*pi/180

    Radius = 6371
    haversine = sin((lat2-lat1)/2)**2 + cos(lat1)*cos(lat2) * sin((long2-long1)/2)**2
    c = 2*atan2(sqrt(haversine), sqrt(1-haversine))
    dist = Radius*c
    return dist



In [112]:
#CBG shapefiles, to find them go to https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html , find the year you want and under Download choose
# "FTP Archive by State"


#Make sure shapefiles agree with safegraph by creating the map with POI's included

#2022 Shapefiles
# gdf1 = gpd.read_file('tl_rd22_24_bg/tl_rd22_24_bg.shp')
# gdf2 = gpd.read_file('tl_rd22_42_bg/tl_rd22_42_bg.shp')
# gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))


#2010 Shapefiles use these for safegraph (safegraph data is from 2021 but they use 2010 cbgs)






In [113]:
def gen_graph(df, weighted=True):
    """
    Reads from a SafeGraph dataframe of a certain location.

    Args:
        df (pd.DataFrame): The SafeGraph dataset of a particular location.
        weighted (bool, optional): Whether the result should have weighted edges. Defaults to True.

    Returns:
        nx.Graph: Graph where nodes are the CBGs, and edges and weights are determined by visitor interactions.
    """
    G = nx.Graph()
    regg_count = 0  # Counter for rows with errors
    
    for _, row in df.iterrows():
        # Extract and process 'poi_cbg'
        poi_cbg = str(row['poi_cbg'])
        try:
            poi_cbg = str(int(float(poi_cbg)))
        except ValueError:
            regg_count += 1
            continue
        
        # Add the 'poi_cbg' node with 'pois' attribute
        if not G.has_node(poi_cbg):
            G.add_node(poi_cbg, pois=[])
        G.nodes[poi_cbg]['pois'].append(row['safegraph_place_id'])
        
        # Set weight (use 1 if unweighted)
        weight = row['median_dwell'] if weighted and 'median_dwell' in row else 1
        
        # Process visitor CBGs
        for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
            visitor_cbg = str(int(float(visitor_cbg)))  # Convert visitor_cbg to a consistent format
            
            if visitor_cbg == poi_cbg:
                continue  # Skip self-loops
            
            # Add or update edge weights
            if G.has_edge(visitor_cbg, poi_cbg):
                G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)
            else:
                G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight))])
        
        # Remove nodes without edges
        if G.degree[poi_cbg] == 0:
            G.remove_node(poi_cbg)
    
    # Convert to undirected graph and sum bidirectional weights
    UG = nx.Graph()
    for u, v, data in G.edges(data=True):
        weight = data['weight']
        if UG.has_edge(u, v):
            UG[u][v]['weight'] += weight
        else:
            UG.add_edge(u, v, weight=weight)
    
    print(f"G has {nx.number_of_nodes(UG)} nodes and {nx.number_of_edges(UG)} edges.")
    print("Bad data count:", regg_count)
    
    return UG


In [114]:
def gen_graph(df, weighted=True):
    """
    Reads from a SafeGraph dataframe of a certain location.

    Args:
        df (pd.DataFrame): The SafeGraph dataset of a particular location.
        weighted (bool, optional): Whether the result should have weighted edges. Defaults to True.

    Returns:
        nx.Graph: Graph where nodes are the CBGs, and edges and weights are determined by visitor interactions.
    """
    G = nx.Graph()
    regg_count = 0  # Counter for rows with errors
    
    for _, row in df.iterrows():
        # Extract and process 'poi_cbg'
        poi_cbg = str(row['poi_cbg'])
        try:
            poi_cbg = str(int(float(poi_cbg)))
        except ValueError:
            regg_count += 1
            continue
        
        # Add the 'poi_cbg' node with 'pois' attribute
        if not G.has_node(poi_cbg):
            G.add_node(poi_cbg, pois=[])
        G.nodes[poi_cbg]['pois'].append(row['safegraph_place_id'])
        
        # Set weight (use 1 if unweighted)
        weight = row['median_dwell'] if weighted and 'median_dwell' in row else 1
        
        # Check if 'visitor_daytime_cbgs' is a string before parsing
        if isinstance(row['visitor_daytime_cbgs'], str):
            try:
                visitor_data = json.loads(row['visitor_daytime_cbgs'])
            except json.JSONDecodeError:
                regg_count += 1
                continue
        else:
            regg_count += 1
            continue  # Skip rows where 'visitor_daytime_cbgs' is not a string

        # Ensure visitor_data is a dictionary
        if not isinstance(visitor_data, dict):
            regg_count += 1
            continue

        # Process visitor CBGs
        for visitor_cbg, num_visitors in visitor_data.items():
            visitor_cbg = str(int(float(visitor_cbg)))  # Convert visitor_cbg to a consistent format
            
            if visitor_cbg == poi_cbg:
                continue  # Skip self-loops
            
            # Add or update edge weights
            if G.has_edge(visitor_cbg, poi_cbg):
                G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)
            else:
                G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight))])
        
        # Remove nodes without edges
        if G.degree[poi_cbg] == 0:
            G.remove_node(poi_cbg)
    
    # Convert to undirected graph and sum bidirectional weights
    UG = nx.Graph()
    for u, v, data in G.edges(data=True):
        weight = data['weight']
        if UG.has_edge(u, v):
            UG[u][v]['weight'] += weight
        else:
            UG.add_edge(u, v, weight=weight)
    
    print(f"G has {nx.number_of_nodes(UG)} nodes and {nx.number_of_edges(UG)} edges.")
    print("Bad data count:", regg_count)
    
    return UG


In [115]:
#get population of a cbg
cbg_pops = pd.read_csv('safegraph_cbg_population_estimate.csv', index_col='census_block_group')


def cbg_population(cbg):
    try:
        return int(cbg_pops.loc[int(cbg)].B00002e1 * 24)
    except:
        return 1000

print(cbg_population(core_cbg))

1416


In [ ]:
#get coordinates
gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
gdf['coords'] = [coords[0] for coords in gdf['coords']]
gdf['longitude'] = gdf['coords'].apply(lambda x: x[0])
gdf['latitude'] = gdf['coords'].apply(lambda x: x[1])
print('creating graph')


#To create regular graph
G = gen_graph(df)



#To create distance graph (save this graph, it takes long to create)

#G = gen_d_graph(df,gdf) #for distance graph or just load (below)



#Save graph

#with open("dist_graph.pkl", "wb") as file:
    #pickle.dump(G, file)

#Load graph
#with open("dist_graph.pkl", "rb") as file:
    #G = pickle.load(file)



creating graph


In [118]:
def gen_graph(df, weighted=True):
    """
    Reads from a SafeGraph dataframe of a certain location.

    Args:
        df (pd.DataFrame): The SafeGraph dataset of a particular location.
        weighted (bool, optional): Whether the result should have weighted edges. Defaults to True.

    Returns:
        nx.Graph: Graph where nodes are the CBGs, and edges and weights are determined by visitor interactions.
    """
    G = nx.Graph()
    regg_count = 0  # Counter for rows with errors
    
    for _, row in df.iterrows():
        # Extract and process 'poi_cbg'
        poi_cbg = str(row['poi_cbg'])
        try:
            poi_cbg = str(int(float(poi_cbg)))
        except ValueError:
            regg_count += 1
            continue
        
        # Add the 'poi_cbg' node with 'pois' attribute
        if not G.has_node(poi_cbg):
            G.add_node(poi_cbg, pois=[])
        G.nodes[poi_cbg]['pois'].append(row['safegraph_place_id'])
        
        # Set weight (use 1 if unweighted)
        weight = row['median_dwell'] if weighted and 'median_dwell' in row else 1
        
        # Process visitor CBGs
        for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
            visitor_cbg = str(int(float(visitor_cbg)))  # Convert visitor_cbg to a consistent format
            
            if visitor_cbg == poi_cbg:
                continue  # Skip self-loops
            
            # Add or update edge weights
            if G.has_edge(visitor_cbg, poi_cbg):
                G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)
            else:
                G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight))])
        
        # Remove nodes without edges
        if G.degree[poi_cbg] == 0:
            G.remove_node(poi_cbg)
    
    # Convert to undirected graph and sum bidirectional weights
    UG = nx.Graph()
    for u, v, data in G.edges(data=True):
        weight = data['weight']
        if UG.has_edge(u, v):
            UG[u][v]['weight'] += weight
        else:
            UG.add_edge(u, v, weight=weight)
    
    print(f"G has {nx.number_of_nodes(UG)} nodes and {nx.number_of_edges(UG)} edges.")
    print("Bad data count:", regg_count)
    
    return UG


In [119]:
#To save graph as csv
import pandas as pd
import numpy as np

def graph_to_csv(UG, filename='adj_graph.csv'):
    #Get adjacency matrix
    adj_matrix = nx.adjacency_matrix(UG)

    array_adj_matrix = adj_matrix.toarray()

    # Create the dataframe
    nodes = list(UG.nodes())
    df_adj_matrix = pd.DataFrame(array_adj_matrix, index=nodes, columns=nodes)

    
    df_adj_matrix.to_csv(filename)

    print('Graph written')




In [120]:
#TEST
"""
import geopandas as gpd
from shapely.geometry import Point

# Assuming 'gdf' is a GeoDataFrame containing CBG geometries and 'GEOID10' identifiers
# Let's say we have the latitude and longitude for Frederick, MD
frederick_md_lat = 39.4143
frederick_md_lon = -77.4105
frederick_point = Point(frederick_md_lon, frederick_md_lat)

# Find the GEOID10 for the CBG that contains the Frederick, MD point
contains_frederick = gdf.contains(frederick_point)
frederick_geoid10 = gdf[contains_frederick]['GEOID10'].values[0]

print("The GEOID10 for Frederick, MD is:", frederick_geoid10)
"""
#240217502002 Frederick
cbg_geocode(core_cbg)

C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1106654488.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = cbg_row.geometry.centroid.iloc[0]


(39.64007694608107, -77.70954929430157)

In [121]:
#Get total population
total_cbgs_population = 0
for cbg in df.poi_cbg.drop_duplicates().tolist():
   
    try:
        cbg_value = df.loc[cbg]
        # Process the cbg_value as needed, e.g., add to total population
        total_cbgs_population += cbg_value
    except KeyError:
        # If KeyError is encountered, handle it here (e.g., skip or log the missing cbg)
        print(f"CBG {cbg} not found in DataFrame, skipping.")
        
print(total_cbgs_population)

CBG 240430006011 not found in DataFrame, skipping.
CBG 240430009003 not found in DataFrame, skipping.
CBG 240430005001 not found in DataFrame, skipping.
CBG 240430104003 not found in DataFrame, skipping.
CBG 240430006021 not found in DataFrame, skipping.
CBG 240430104004 not found in DataFrame, skipping.
CBG 240430007003 not found in DataFrame, skipping.
CBG 240430006022 not found in DataFrame, skipping.
CBG 240430103003 not found in DataFrame, skipping.
CBG 240430010014 not found in DataFrame, skipping.
CBG 240430112013 not found in DataFrame, skipping.
CBG 240430102002 not found in DataFrame, skipping.
CBG 240430005002 not found in DataFrame, skipping.
CBG 240430010011 not found in DataFrame, skipping.
CBG 240430008003 not found in DataFrame, skipping.
CBG 240430103001 not found in DataFrame, skipping.
CBG 240430010013 not found in DataFrame, skipping.
CBG 240430102003 not found in DataFrame, skipping.
CBG 240430111003 not found in DataFrame, skipping.
CBG 240430102004 not found in D

In [122]:
def find_adjacent_cbgs(gdf, current_cbg):
    """Find directly adjacent CBGs to the current CBG."""
    current_geom = gdf.loc[gdf['GEOID10'] == current_cbg, 'geometry'].values[0]
    neighbors = gdf[gdf.touches(current_geom)]['GEOID10'].tolist()
    return neighbors

def create_cluster(gdf, core_cbg, min_pop):
    visited = {core_cbg}
    queue = [core_cbg]
    cluster = []
    total_population = 0
    
    while queue and total_population < min_pop:
        current_cbg = queue.pop(0)
        cluster.append(current_cbg)
        total_population += cbg_population(current_cbg)
        
        if total_population >= min_pop:
            break
        
        neighbors = find_adjacent_cbgs(gdf, current_cbg)
        for neighbor in neighbors:
            if neighbor not in visited:
                queue.append(neighbor)
                visited.add(neighbor)
                
    return cluster, total_population

In [123]:
def greedy_weight(C, u0, min_pop):
    
    cluster = [u0]
    population = cbg_population(u0)
    it = 1
    
    while population < min_pop:
        # Get all adjacent CBGs of the cluster
        all_adj_cbgs = []
        for i in cluster:
            adj_cbgs = list(C.adj[i])
            for j in adj_cbgs:
                if j not in all_adj_cbgs and j not in cluster:
                    all_adj_cbgs.append(j)
        
        max_movement = 0
        cbg_to_add = 0
        
        # Calculate the movement from S to all adjacent CBGs
        for i in all_adj_cbgs:
            current_movement = 0
            for j in cluster:
                if j in C.adj[i]:  # Check if the edge exists
                    current_movement += C.adj[i][j].get('weight', 0)
            
            # Find the CBG with the greatest movement and add it to S
            if current_movement > max_movement:
                max_movement = current_movement
                cbg_to_add = i
        
        # Add selected CBG to cluster and update population
        cluster.append(cbg_to_add)
        print("  Prev pop  ", population, end="", flush=True)
        population += cbg_population(cbg_to_add)
        print("  New pop  ", population, end="", flush=True)
    
    return cluster, population
            
        
        
    
   

        
                
       

In [124]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import folium
from folium import plugins
from folium import Marker

In [125]:

print('starting algo')
algorithm_result = greedy_weight(G, core_cbg, 100000)
print('algo done')
print('cluster population:', algorithm_result[1], '# of cbgs:', len(algorithm_result[0]))
print(algorithm_result[0])
movement_in_cbgs = 0
movement_out_cbgs = 0


for i in algorithm_result[0]:
    adj = list(G.adj[i])
    for j in adj:
        if j in algorithm_result[0]:
            movement_in_cbgs += G.adj[i][j]['weight']/2
        else:
            movement_out_cbgs += G.adj[i][j]['weight']
    

print("IN ", movement_in_cbgs, "OUT", movement_out_cbgs, " Ratio ", movement_in_cbgs/(movement_in_cbgs+movement_out_cbgs) )
"""

# Load and prepare data, including the GeoDataFrame `gdf`

# Define core CBG and population requirements
core_cbg = '401091032001'  # Core CBG for the cluster
min_cluster_pop = 100000  # Minimum population for the cluster

# Generate the cluster using the new method
print('Starting new clustering algorithm...')
cluster_cbgs, cluster_population = create_cluster(gdf, core_cbg, min_cluster_pop)
print('Algorithm done')
print(f'Cluster population: {cluster_population}, # of CBGs: {len(cluster_cbgs)}')
print(cluster_cbgs)

# Calculate movement metrics within the newly generated cluster
movement_in_cbgs = 0
movement_out_cbgs = 0

# Assuming G is your graph generated by gen_graph or gen_d_graph function
for cbg in cluster_cbgs:
    if cbg in G:  # Ensure CBG is a node in the graph
        adj = list(G.adj[cbg])
        for j in adj:
            if j in cluster_cbgs:
                movement_in_cbgs += G[cbg][j]['weight'] / 2
            else:
                movement_out_cbgs += G[cbg][j]['weight']

print(f"IN: {movement_in_cbgs}, OUT: {movement_out_cbgs}, Ratio: {movement_in_cbgs / (movement_in_cbgs + movement_out_cbgs)}")

"""        
    
    
    

starting algo


KeyError: '240430006012'

In [ ]:
from shapely.geometry import Point
import geopandas as gpd

# Load your GeoDataFrame containing the CBG geometries here if not already loaded
# gdf = gpd.read_file('path_to_your_shapefile.shp')

# Step 1: Define the geographical center of Oklahoma City (using the Oklahoma City National Memorial & Museum as a central point)
okc_center = Point(-75.1652, 39.9526)

# Step 2: Find the CBG containing this point
cbg_center = gdf[gdf.geometry.contains(okc_center)]
print(cbg_center['GEOID10'])



12757    421010005001
Name: GEOID10, dtype: object


In [ ]:

def initialize_map(seed_geocode):
    #seed = cbg_geocode(240430006012)
    #center = (-97.5185, 35.4576)
    center = (-76.6122, 39.2904)
    return folium.Map(location=[center[1],center[0]], zoom_start=13)

def find_neighbors(gdf, cbgs):
    cbg_n = {}
    for cbg in cbgs:
        try:
            row = gdf[gdf['GEOID10'] == cbg].iloc[0]
        except IndexError:
            continue
        current_neighbours = gdf[gdf.geometry.touches(row['geometry'])]['GEOID10']
        for i in current_neighbours:
            if i not in cbgs:
                cbg_n[i] = cbg_n.get(i, 0) + 1
    return cbg_n

def mark_black_cbgs(gdf, cbg_n, map, algo_copy, black_cbgs):
    for key, value in cbg_n.items():
        if value >= 2 and ((key not in algo_copy) and (key not in black_cbgs)):  
            print(f'found: {key}\n')
            shape = gdf[gdf['GEOID10'] == key]
            layer = folium.GeoJson(shape, style_function=lambda feature: {
                'fillColor': '#000000', 'color': '#000000', 'fillOpacity': 0.4}).add_to(map)
            layer.add_child(folium.Popup(key))
            algo_copy.append(key)
            black_cbgs.append(key)

def remove_unnecessary_cbg(gdf, black_cbgs, algo_copy):
    to_remove = []
    for cbg in black_cbgs:
        row = gdf[gdf['GEOID10'] == cbg].iloc[0]
        neighbors = gdf[gdf.geometry.touches(row['geometry'])]['GEOID10'].tolist()
        outside_borders = sum(1 for neighbor in neighbors if neighbor not in algo_copy and neighbor not in black_cbgs)
        if outside_borders >= 2:
            to_remove.append(cbg)
    for cbg in to_remove:
        black_cbgs.remove(cbg)
        algo_copy.remove(cbg)

def save_map(map):
    print('saving')
    map.save("black_clusters.html")

map = initialize_map(core_cbg)
algo_copy = algorithm_result[0].copy()
black_cbgs = []

for _ in range(5):
    cbg_n = find_neighbors(gdf, algo_copy)
    mark_black_cbgs(gdf, cbg_n, map, algo_copy, black_cbgs)

remove_unnecessary_cbg(gdf, black_cbgs, algo_copy)

save_map(map)




found: 245101205002

found: 245101001002

found: 245101003001

found: 245101001004

found: 245100604002

found: 245100301002

found: 245100202001

found: 245100704003

found: 245100704001

found: 245101202022

found: 245101201004

found: 245101202024

found: 245101201001

found: 245101207002

found: 245101206001

found: 245102201001

found: 245101102002

found: 245102201002

found: 245102201003

found: 245101701002

found: 245101803002

found: 245101801001

found: 245101902002

found: 245102503031

found: 245102607002

found: 245102605013

found: 245102604041

found: 245102609003

found: 245102608001

found: 245102605012

found: 245102709032

found: 245102710021

found: 245102709033

found: 245102710012

found: 245102709011

found: 245101702003

found: 245101205001

found: 245101401004

found: 245101101001

found: 245102401002

found: 245102606051

found: 245102606043

found: 245100101003

found: 245100104002

found: 245101202012

found: 245101203001

found: 245100905001

found: 245100

In [ ]:
#Map final cbg states as animation - NEEDED
import random

final_cbgs = []
movement_in_S = 0
movement_out_S = 0
colors = ['#008000', '#FFFF00', '#FFA500', '#FF0000']

center = (-76.6122, 39.2904)
map = folium.Map(location=[center[1], center[0]], zoom_start=13)


def calculate_opacity_and_color(ratio, distance):
    rand = random.randint(1,10)

    """
    Calculate opacity and color based on movement ratio and distance from the center.
    
    Args:
        ratio (float): Ratio of people moving in vs. total movement.
        distance (float): Distance from the city center.

    Returns:
        tuple: (color, opacity) where color is a hex string and opacity is a float.
    """
    # Set color based on ratio
    color = '#FF0000'  
    # Set opacity inversely proportional to distance (closer = higher opacity)
    opacity = max(0.2, 1.0 - min(distance / 10.0, 0.8))  # Scale distance to opacity
    if rand > 9:
        opacity = max(distance / 10.0, 0.7)
    return color, opacity

# Prepare the initial features list for regular CBGs
features = []
# Process the algorithm result CBGs
for i, cbg in enumerate(algorithm_result[0]):
    movement_in_S = 0
    movement_out_S = 0
    shape = gdf[gdf['GEOID10'] == cbg]

    edges = G.edges(cbg)
    for j in edges:
        if j[1] in algorithm_result[0]:
            movement_in_S += G[cbg][j[1]]['weight'] / 2
        else:
            movement_out_S += G[cbg][j[1]]['weight']

    ratio = movement_in_S / (movement_in_S + movement_out_S) if (movement_in_S + movement_out_S) > 0 else 0

    # Get coordinates for the CBG
    cbg_coords = cbg_geocode(cbg)
    if cbg_coords is None:
        print(f"Warning: No coordinates found for CBG {cbg}, skipping.")
        continue  # Skip this CBG if coordinates are not found

    # Calculate distance from the center
    distance_from_center = distance(center[1], center[0], cbg_coords[0], cbg_coords[1])

    # Determine color and opacity based on ratio and distance
    fillcolor, fillopacity = calculate_opacity_and_color(ratio, distance_from_center)

    if not shape.empty:
        shape = shape.to_crs("EPSG:4326")
        geojson = json.loads(shape.to_json())
        feature = geojson['features'][0]
        feature['properties']['times'] = [(pd.Timestamp('today') + pd.Timedelta(i, 'H')).isoformat()]
        feature['properties']['style'] = {'fillColor': fillcolor, 'color': fillcolor, 'fillOpacity': fillopacity}
        features.append(feature)

# Add black CBGs with the same logic
for i, cbg in enumerate(black_cbgs, start=len(algorithm_result[0])):
    shape = gdf[gdf['GEOID10'] == cbg]
    if not shape.empty:
        shape = shape.to_crs("EPSG:4326")
        geojson = json.loads(shape.to_json())
        
        # Assign a default movement ratio for black CBGs
        ratio = 0.5  # Low movement ratio for black CBGs
        
        # Get coordinates and calculate distance from the center
        cbg_coords = cbg_geocode(cbg)
        if cbg_coords is None:
            print(f"Warning: No coordinates found for CBG {cbg}, skipping.")
            continue
        distance_from_center = distance(center[1], center[0], cbg_coords[0], cbg_coords[1])
        
        # Determine color and opacity based on ratio and distance
        fillcolor, fillopacity = calculate_opacity_and_color(ratio, distance_from_center)

        feature = geojson['features'][0]
        feature['properties']['times'] = [(pd.Timestamp('today') + pd.Timedelta(i, 'H')).isoformat()]
        feature['properties']['style'] = {'fillColor': fillcolor, 'color': fillcolor, 'fillOpacity': fillopacity}
        features.append(feature)

# Add the features with timestamps to the map
existing_cluster_layer = plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT1H',  # Update every hour
    add_last_point=True,
    auto_play=False,
    loop=False
)
map.add_child(existing_cluster_layer)

# Save the map
map.save("hagerstown_ratios1.html")

C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1106654488.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = cbg_row.geometry.centroid.iloc[0]
C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1773626371.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  feature['properties']['times'] = [(pd.Timestamp('today') + pd.Timedelta(i, 'H')).isoformat()]
C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1106654488.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = cbg_row.geometry.centroid.iloc[0]
C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1773626371.py:67: FutureWarning: 'H' is deprecated and will be removed in a future vers

CBG code 245101000000 not found in the data.
CBG code 245103000000 not found in the data.
CBG code 245100000000 not found in the data.
CBG code 245102000000 not found in the data.


C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1106654488.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = cbg_row.geometry.centroid.iloc[0]
C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1773626371.py:67: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  feature['properties']['times'] = [(pd.Timestamp('today') + pd.Timedelta(i, 'H')).isoformat()]
C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1106654488.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = cbg_row.geometry.centroid.iloc[0]
C:\Users\keeya\AppData\Local\Temp\ipykernel_3960\1773626371.py:67: FutureWarning: 'H' is deprecated and will be removed in a future vers

In [ ]:

"""
import pandas as pd

# Load the CSV file
df = pd.read_csv(location_name + '.csv')

# Convert the poi_cbg column to string if it's not already, to ensure correct matching
df['poi_cbg'] = df['poi_cbg'].astype(str)

# Filter rows where the poi_cbg column is in the final_cbgs list
filtered_df = df[df['poi_cbg'].isin(final_cbgs)]

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv(location_name + '_cluster_data.csv', index=False)

# Get unique CBG values from the filtered DataFrame
unique_cbgs = filtered_df['poi_cbg'].drop_duplicates()

# Create a DataFrame from these unique CBGs
unique_cbgs_df = pd.DataFrame(unique_cbgs)

# Save the DataFrame of unique CBGs to a new CSV file
unique_cbgs_df.to_csv(location_name + '_cluster_cbgs.csv', index=False)
"""

import geopandas as gpd
import folium
from geopy.geocoders import Nominatim

def get_lat_lon(city_name):

    geolocator = Nominatim(user_agent="cbg_finder")
    location = geolocator.geocode(city_name)
    
    if location:
        return location.latitude, location.longitude
    else:
        raise Exception(f"City {city_name} not found.")

def find_core_cbg(city_name, gdf):

    lat, lon = get_lat_lon(city_name)
    
    # Buffer point to slightly adjust the distance for a rough match within a few meters.
    point = gpd.GeoSeries(gpd.points_from_xy([lon], [lat]), crs="EPSG:4326").to_crs(gdf.crs)
    
    # Find the CBG that contains the point
    cbg_row = gdf[gdf.contains(point.geometry[0])]
    
    if not cbg_row.empty:
        return cbg_row.iloc[0]
    else:
        raise Exception(f"No CBG found for {city_name}")

def plot_cbg_on_map(cbg_data, city_name):

    lat, lon = get_lat_lon(city_name)
    
    # Initialize the folium map centered on the city
    city_map = folium.Map(location=[lat, lon], zoom_start=12)
    
    # Highlight the core CBG using __geo_interface__
    folium.GeoJson(
        data=cbg_data['geometry'].__geo_interface__,
        style_function=lambda x: {
            'fillColor': '#FF0000', 'color': 'black', 'fillOpacity': 0.6
        }
    ).add_to(city_map)
    
    # Save the map to an HTML file
    map_filename = f"core_cbgs/{city_name}.html"
    city_map.save(map_filename)
    
    print(f"Map for {city_name} saved as '{map_filename}'")

import geopandas as gpd
import numpy as np

import pandas as pd

import networkx as nx
import json
import pickle

gdf1 = gpd.read_file('tl_2010_24_bg10/tl_2010_24_bg10.shp')
gdf2 = gpd.read_file('tl_2010_42_bg10/tl_2010_42_bg10.shp')
gdf3 = gpd.read_file('tl_2010_40_bg10/tl_2010_40_bg10.shp')
gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2, gdf3], ignore_index=True))

# Example usage: Highlighting core CBG 
city_name = "baltimore"
try:
    core_cbg_data = find_core_cbg(city_name, gdf)
    print(core_cbg_data)
    plot_cbg_on_map(core_cbg_data, city_name)
except Exception as e:
    print(e)


STATEFP10                                                    24
COUNTYFP10                                                  510
TRACTCE10                                                040100
BLKGRPCE10                                                    1
GEOID10                                            245100401001
NAMELSAD10                                        Block Group 1
MTFCC10                                                   G5030
FUNCSTAT10                                                    S
ALAND10                                                  816666
AWATER10                                                   1430
INTPTLAT10                                          +39.2894559
INTPTLON10                                         -076.6130872
geometry      POLYGON ((-76.619197 39.286321, -76.6193689999...
Name: 91, dtype: object
Map for baltimore saved as 'core_cbgs/baltimore.html'
